In [1]:
from cnns_classes import t1_quantizedCNN, t2_quantizedCNN

In [2]:
#https://www.kaggle.com/code/vmarkin/advatt
# carregar as bibliotecas básicas necessárias
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from utils import utils

# gerar os gráficos logo abaixo dos comandos de plot
%matplotlib inline
torch.manual_seed(4)

In [3]:
train_qtdy = 324000
test_qtdy = 81000

train_data_path = './dataset/X_train_sat6.csv'
train_label_path = './dataset/y_train_sat6.csv'
test_data_path = './dataset/X_test_sat6.csv'
test_label_path = './dataset/y_test_sat6.csv'

train_dataset_output_name = "train_dataset.pkl"
test_dataset_output_name = "test_dataset.pkl"

t1_quantizations=[2,4,8]
t2_quantizations=[2,4,8]

device = torch.device('cpu')

In [4]:
def data_read(data_path, nrows):
    data=pd.read_csv(data_path, header=None, nrows=nrows, dtype=np.uint8)
    data=data.values ## converting the data into numpy array
    return data

In [5]:
class SatImgDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.transform = T.ToTensor()
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        x = self.transform(self.X[index])
        y = torch.FloatTensor(self.y[index])
        return {'x':x, 'y':y}

In [6]:
train_data=data_read(train_data_path, nrows=train_qtdy)
print("Train data shape:" + str(train_data.shape))

##Read training data labels
train_data_label=data_read(train_label_path,nrows=train_qtdy)
print("Train data label shape:" + str(train_data_label.shape))

##Read test data
test_data=data_read(test_data_path, nrows=test_qtdy)
print("Test data shape:" + str(test_data.shape))

##Read test data labels
test_data_label=data_read(test_label_path,nrows=test_qtdy)
print("Test data label shape:" + str(test_data_label.shape))

train_data_reshaped = train_data.reshape(train_qtdy,28,28,4)
test_data_reshaped = test_data.reshape(test_qtdy,28,28,4) 

final_train_data = np.zeros((train_qtdy, 32, 32, 4),dtype=np.float32)
final_train_data[:, :28, :28, :] = train_data_reshaped;

final_test_data = np.zeros((test_qtdy, 32, 32, 4),dtype=np.float32)
final_test_data[:, :28, :28, :] = test_data_reshaped;

dataset_train = SatImgDataset(final_train_data, train_data_label)
dataset_test = SatImgDataset(final_test_data, test_data_label)

Train data shape:(324000, 3136)
Train data label shape:(324000, 6)
Test data shape:(81000, 3136)
Test data label shape:(81000, 6)


In [7]:
utils.save_object(object=dataset_train, filename=train_dataset_output_name)

In [8]:
utils.save_object(object=dataset_test, filename=test_dataset_output_name)

In [6]:
dataset1 = utils.read_object(filename=train_dataset_output_name)
print(dataset1)

dataset2 = utils.read_object(filename=test_dataset_output_name)
print(dataset2)